Mesh generation from MHD voxel files using FAST extraction

In [ ]:
from stl import mesh as stmesh
import math
import numpy as np
import fast

indir = "./segmentations/mhd/";
outdir = "./meshes/";
# Spine
#files = [\
#    "vertebrae_L1",\
#    "vertebrae_L2",\
#    "vertebrae_L3",\
#    "vertebrae_L4",\
#    "vertebrae_L5",\
#    "vertebrae_T1",\
#    "vertebrae_T10",\
#    "vertebrae_T11",\
#    "vertebrae_T12",\
#    "vertebrae_T2",\
#    "vertebrae_T3",\
#    "vertebrae_T4",\
#    "vertebrae_T5",\
#    "vertebrae_T6",\
#    "vertebrae_T7",\
#    "vertebrae_T8",\
#    "vertebrae_T9",\
#    "sacrum"];

# Heart
#files = [\
#    "heart_atrium_left",\
#    "heart_atrium_right",\
#    "heart_myocardium",\
#    "heart_ventricle_left",\
#    "heart_ventricle_right"];

# Aorta
files = ["aorta"]

verts = []
faces = []

offset = 0

for file in files:
    importer = fast.ImageFileImporter.create(indir + file + ".mhd")
    smoothing = fast.GaussianSmoothing.create(stdDev=2.0).connect(importer)
    extraction = fast.SurfaceExtraction.create(threshold=50).connect(smoothing)
    mesh = extraction.runAndGetOutputData()
    access = mesh.getMeshAccess(fast.ACCESS_READ)

    for vertex in access.getVertices():
        verts.append(vertex.getPosition().reshape(1, 3))

    for face in access.getTriangles():
        faces.append([offset + face.getEndpoint1(), offset + face.getEndpoint2(), offset + face.getEndpoint3()])
        
    offset = len(verts)
    
    print(file)
    
verts = np.array(verts)
faces = np.array(faces)
    
model = stmesh.Mesh(np.zeros(faces.shape[0], dtype=stmesh.Mesh.dtype))
for i, f in enumerate(faces):
    for j in range(3):
        model.vectors[i][j] = verts[f[j],:]
    
model.save(outdir + "output.stl")

Centerline extraction using VMTK

In [ ]:
from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtkimagereader -ifile ./segmentations/mhd/aorta.mhd --pipe vmtkimagewriter -ofile ./segmentations/vti/aorta.vti"

res = pypes.PypeRun(args)

In [ ]:
# Example marching cubes surface extraction without level set meshing from segmented file

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtkimagereader -ifile ./segmentations/vti/aorta.vti --pipe\
        vmtkmarchingcubes -l 120.0 -ofile ./meshes/vtp/aorta.vtp --pipe\
        vmtkrenderer --pipe vmtkimageviewer --pipe vmtksurfaceviewer"

res = pypes.PypeRun(args)

In [1]:
# Segmenting the aorta via levelsets

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtklevelsetsegmentation -ifile ./segmentations/vti/aorta.vti --pipe vmtkmarchingcubes -i @.o -ofile ./meshes/vtp/aorta.vtk"

pypes.PypeRun(args)


Automatic piping vmtklevelsetsegmentation
Parsing options vmtklevelsetsegmentation
    ImageInputFileName = ./segmentations/vti/aorta.vti
Explicit piping vmtklevelsetsegmentation
Input vmtklevelsetsegmentation members:
    Id = 0
    Disabled = 0
    Image = None
    ImageInputFileName = ./segmentations/vti/aorta.vti
    FeatureImage = None
    FeatureImageInputFileName = 
    InitializationImage = None
    InitializationImageInputFileName = 
    InitialLevelSets = None
    InitialLevelSetsInputFileName = 
    LevelSets = None
    LevelSetsInputFileName = 
    LevelSetsType = geodesic
    FeatureImageType = gradient
    NegateForInitialization = 0
    SigmoidRemapping = 0
    IsoSurfaceValue = 0.0
    DerivativeSigma = 0.0
    FeatureDerivativeSigma = 0.0
    UpwindFactor = 1.0
    FWHMRadius = [1.0, 1.0, 1.0]
    FWHMBackgroundValue = 0.0
    NumberOfIterations = 0
    PropagationScaling = 0.0
    CurvatureScaling = 0.0
    AdvectionScaling = 1.0
    EdgeWeight = 0.0
    SmoothingIte

In [2]:
# Clipping mesh inlets and outlets
  
from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfaceclipper -ifile ./meshes/vtp/aorta.vtk -ofile ./meshes/vtp/aorta_clip.vtk"

res = pypes.PypeRun(args)


Automatic piping vmtksurfaceclipper
Parsing options vmtksurfaceclipper
    SurfaceInputFileName = ./meshes/vtp/aorta.vtk
    SurfaceOutputFileName = ./meshes/vtp/aorta_clip.vtk
Explicit piping vmtksurfaceclipper
Input vmtksurfaceclipper members:
    Id = 0
    Disabled = 0
    Surface = None
    SurfaceInputFileName = ./meshes/vtp/aorta.vtk
    WidgetType = box
    Transform = None
    CleanOutput = 1
    InsideOut = 0
    Interactive = 1
    ClipArrayName = None
    ClipValue = 0.0
    vmtkRenderer = None
    SurfaceOutputFileName = ./meshes/vtp/aorta_clip.vtk
    ClippedSurfaceOutputFileName = 
    CutLinesOutputFileName = 
Reading VTK surface file.
Executing vmtksurfaceclipper ...
Quit renderer
Done executing vmtksurfaceclipper.
Writing VTK surface file.
Output vmtksurfaceclipper members:
    Id = 0
    Surface = vtkPolyData
    ClippedSurface = vtkPolyData
    CutLines = vtkPolyData
    Transform = vtkTransform


In [3]:
# Surface smoothing

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfacesmoothing -ifile ./meshes/vtp/aorta_clip.vtk -passband 0.1 -iterations 30 -ofile ./meshes/vtp/aorta_smooth.vtk "

res = pypes.PypeRun(args)


Automatic piping vmtksurfacesmoothing
Parsing options vmtksurfacesmoothing
    SurfaceInputFileName = ./meshes/vtp/aorta_clip.vtk
    NumberOfIterations = 30
    PassBand = 0.1
    SurfaceOutputFileName = ./meshes/vtp/aorta_smooth.vtk
Explicit piping vmtksurfacesmoothing
Input vmtksurfacesmoothing members:
    Id = 0
    Disabled = 0
    Surface = None
    SurfaceInputFileName = ./meshes/vtp/aorta_clip.vtk
    NumberOfIterations = 30
    Method = taubin
    PassBand = 0.1
    RelaxationFactor = 0.01
    BoundarySmoothing = 1
    NormalizeCoordinates = 1
    SurfaceOutputFileName = ./meshes/vtp/aorta_smooth.vtk
Reading VTK surface file.
Executing vmtksurfacesmoothing ...
Done executing vmtksurfacesmoothing.
Writing VTK surface file.
Output vmtksurfacesmoothing members:
    Id = 0
    Surface = vtkPolyData


In [4]:
# Surface subdivision

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfacesubdivision -ifile ./meshes/vtp/aorta_smooth.vtk -ofile ./meshes/vtp/aorta_subdiv.vtk -method butterfly"

res = pypes.PypeRun(args)


Automatic piping vmtksurfacesubdivision
Parsing options vmtksurfacesubdivision
    SurfaceInputFileName = ./meshes/vtp/aorta_smooth.vtk
    Method = butterfly
    SurfaceOutputFileName = ./meshes/vtp/aorta_subdiv.vtk
Explicit piping vmtksurfacesubdivision
Input vmtksurfacesubdivision members:
    Id = 0
    Disabled = 0
    Surface = None
    SurfaceInputFileName = ./meshes/vtp/aorta_smooth.vtk
    NumberOfSubdivisions = 1
    Method = butterfly
    SurfaceOutputFileName = ./meshes/vtp/aorta_subdiv.vtk
Reading VTK surface file.
Executing vmtksurfacesubdivision ...
Done executing vmtksurfacesubdivision.
Writing VTK surface file.
Output vmtksurfacesubdivision members:
    Id = 0
    Surface = vtkPolyData


In [5]:
# Comparison

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfacereader -ifile ./meshes/vtp/aorta_clip.vtk --pipe vmtksurfacesmoothing -iterations 30 -passband 0.1 --pipe vmtkrenderer --pipe vmtksurfaceviewer -display 0 --pipe vmtksurfaceviewer -i @vmtksurfacereader.o -color 1 0 0 -display 1"

res = pypes.PypeRun(args)


Automatic piping vmtksurfacereader
Parsing options vmtksurfacereader
    InputFileName = ./meshes/vtp/aorta_clip.vtk
Explicit piping vmtksurfacereader
Input vmtksurfacereader members:
    Id = 0
    Disabled = 0
    Format = 
    GuessFormat = 1
    Surface = 0
    InputFileName = ./meshes/vtp/aorta_clip.vtk
    SurfaceOutputFileName = 
Executing vmtksurfacereader ...
Reading VTK surface file.
Done executing vmtksurfacereader.
Output vmtksurfacereader members:
    Id = 0
    Surface = vtkPolyData

Automatic piping vmtksurfacesmoothing
    Surface = vmtksurfacereader-0.Surface
Parsing options vmtksurfacesmoothing
    NumberOfIterations = 30
    PassBand = 0.1
Explicit piping vmtksurfacesmoothing
Input vmtksurfacesmoothing members:
    Id = 0
    Disabled = 0
    Surface = vtkPolyData
    SurfaceInputFileName = 
    NumberOfIterations = 30
    Method = taubin
    PassBand = 0.1
    RelaxationFactor = 0.01
    BoundarySmoothing = 1
    NormalizeCoordinates = 1
    SurfaceOutputFileName =

In [ ]:
# Centerline computation

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtkcenterlines -ifile ./meshes/vtp/aorta_smooth.vtp -ofile ./meshes/vtp/aorta_centerline.vtp"

res = pypes.PypeRun(args)

In [ ]:
# Centerline computation visualization

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfacereader -ifile ./meshes/vtp/aorta_smooth.vtp --pipe vmtkcenterlines --pipe vmtkrenderer --pipe vmtksurfaceviewer -opacity 0.25 --pipe vmtksurfaceviewer -i @vmtkcenterlines.o -array MaximumInscribedSphereRadius"

res = pypes.PypeRun(args)

In [ ]:
# Centerline computation visualization

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtksurfacereader -ifile ./meshes/vtp/aorta_smooth.vtp --pipe vmtkcenterlines --pipe vmtkrenderer --pipe vmtksurfaceviewer -opacity 0.25 --pipe vmtksurfaceviewer -i @vmtkcenterlines.voronoidiagram -array MaximumInscribedSphereRadius --pipe vmtksurfaceviewer -i @vmtkcenterlines.o"

res = pypes.PypeRun(args)

Centerline extraction using skeletonization with voxel structures

In [ ]:
# Alternative voxel-based centerline extraction

import SimpleITK as sitk
import os
import numpy as np
import time
from scipy.ndimage import binary_erosion

file = "./segmentations/mhd/aorta.mhd"

img = sitk.ReadImage(file)

array = sitk.GetArrayFromImage(img)

eroded = array

start = time.perf_counter()

print(np.max(array))

for i in range(0, 10):
    eroded = binary_erosion(eroded)
    array += eroded
    
end = time.perf_counter()

maxval = np.max(array)

print(end - start)
print(maxval)

centerline = []

size = array.shape

for x in range(0, size[0]):
    for y in range(0, size[1]):
        for z in range(0, size[2]):
            if (array[x][y][z] == maxval):
                centerline.append((x, y, z))

Mesh extraction using GMSH from prepared VTK surface mesh

In [5]:
dir(fast.VTKMeshFileImporter)

['New',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__swig_destroy__',
 '__weakref__',
 'attributesToString',
 'connect',
 'create',
 'disableRuntimeMeasurements',
 'enableRuntimeMeasurements',
 'getAllRuntimes',
 'getAttribute',
 'getAttributes',
 'getDataStream',
 'getExecuteOnLastFrameOnly',
 'getFilename',
 'getInputPort',
 'getLastExecuteToken',
 'getMainDevice',
 'getNameOfClass',
 'getNrOfInputConnections',
 'getNrOfInputPorts',
 'getNrOfOutputPorts',
 'getOutputData',
 'getOutputPort',
 'getReporter',
 'getRuntime',
 'getRuntimeManager',
 'getStaticNameOfClass',
 'hasReceivedLastFrameFlag',
 'loadAttributes',
 'run',
 'runAndGetOutputData',
 'setAttributes',
 'setDevice',

In [6]:
import os
import vtk

outdir = "./meshes/";

# Aorta
filepath = "./meshes/vtp/aorta_smooth.vtk"

verts = []
faces = []

offset = 0

basename = os.path.basename(filepath)
print("Copying file:", basename)
basename = os.path.splitext(basename)[0]
outfile = os.path.join(outdir, basename+".stl")
reader = vtk.vtkGenericDataObjectReader()
reader.SetFileName(filepath)
reader.Update()
writer = vtk.vtkSTLWriter()
writer.SetInputConnection(reader.GetOutputPort())
writer.SetFileName(outfile)
writer.Write()

Copying file: aorta_smooth.vtk


1

In [10]:
import gmsh
import sys
import os
import math

gmsh.initialize(sys.argv)

# merge STL, create surface patches that are reparametrizable (so we can remesh
# them) and compute the parametrizations
gmsh.merge(os.path.join('./meshes/.stl'))
gmsh.model.mesh.classifySurfaces(math.pi, True, True)
gmsh.model.mesh.createGeometry()

# make extrusions only return "top" surfaces and volumes, not lateral surfaces
gmsh.option.setNumber('Geometry.ExtrudeReturnLateralEntities', 0)

# extrude a boundary layer of 4 elements using mesh normals (thickness = 0.5)
gmsh.model.geo.extrudeBoundaryLayer(gmsh.model.getEntities(2), [4], [1.0], True)

# extrude a second boundary layer in the opposite direction (note the `second ==
# True' argument to distinguish it from the first one)
e = gmsh.model.geo.extrudeBoundaryLayer(gmsh.model.getEntities(2), [4], [-1.0],
                                        True, True)

# get "top" surfaces created by extrusion
top_ent = [s for s in e if s[0] == 2]
top_surf = [s[1] for s in top_ent]

# get boundary of top surfaces, i.e. boundaries of holes
gmsh.model.geo.synchronize()
bnd_ent = gmsh.model.getBoundary(top_ent)
bnd_curv = [c[1] for c in bnd_ent]

# create plane surfaces filling the holes
loops = gmsh.model.geo.addCurveLoops(bnd_curv)
for l in loops:
    top_surf.append(gmsh.model.geo.addPlaneSurface([l]))

# create the inner volume
gmsh.model.geo.addVolume([gmsh.model.geo.addSurfaceLoop(top_surf)])
gmsh.model.geo.synchronize()

# use MeshAdapt for the resulting not-so-smooth parametrizations
gmsh.option.setNumber('Mesh.Algorithm', 1)
gmsh.option.setNumber('Mesh.MeshSizeFactor', 0.1)

if '-nopopup' not in sys.argv:
    gmsh.fltk.run()

gmsh.finalize()
